# POINTOUT

## Downloading the dataset

The first thing we have to do is go to [POINTOUT](https://targetdetection.com) and download a dataset. In the example below, I download the airplanes dataset in Barcelona's airport.

![gif](https://res.cloudinary.com/dwhsm5imq/image/upload/v1559811423/download_kg0sim.gif)

You can download different datasets on different areas of interest (even in the entire world !). Also, if you are interested in some dataset that does not exist or has a small ammount of annotations you can create a new one and start adding annotations.

Once you download the dataset, you have to copy the *dataset.zip* file alongside this notebook. Also, we need the *utils.py* file. If you are in colab you can get it with the following command. Also, make sure to select the runtime to GPU to share the data with the training notebook.

In [ ]:
# download utils.py (useful in colab)

#!wget https://raw.githubusercontent.com/juansensio/pointout/master/ipynb/utils.py

In [ ]:
# If everything is correct, you should see the file dataset.zip and utils.py here

!ls

## Data exploration

In this section we are going to explore the dataset and transform it to use it later for training of our model.

First, unzip the folder and explore the images and annotations file.

In [ ]:
# unzip dataset to a folder called dataset

import os
import shutil

PATH = 'dataset'

if os.path.isdir(PATH):
    shutil.rmtree(PATH)

!unzip dataset.zip -d {PATH}

In [ ]:
# list files in dataset

!ls {PATH}

In [ ]:
# get images

images = [file for file in os.listdir(PATH) if file[-3:] == 'png']
print('Number of images: ', len(images))

In [ ]:
# visualize some images

from utils import *
import random
import matplotlib.pyplot as plt
%matplotlib inline

img = open_image(PATH + '/' + images[random.randint(0, len(images)-1)])
plt.imshow(img)
print("Image dimensions (h, w, c): ", img.shape)

We can see that images can be very different in size due to the collaborative nature of the platform (each user can make annotations in different sizes).

In [ ]:
# parse annotations

import json
from pprint import pprint

annotations_file_name = '{}/annotations.json'.format(PATH)
with open(annotations_file_name) as f:
    annotations_file = json.load(f)
    
annotations_file[random.randint(0, len(annotations_file)-1)]

The annotations file contains a list of all the images, and for each one it contains some information (the bounding box in map coordinates, the zoom level at which it was saved, ...) and a list of all the annotations.

In [ ]:
# visualize images with annotations

ix = random.randint(0, len(annotations_file)-1)
item = annotations_file[ix]

img_name = item["name"]
anns = item['annotations']
mBB = item['mapBB']
fig, ax = plt.subplots(figsize=(10,10))
img = open_image('{}/{}'.format(PATH,img_name))
ax.set_title('{} {}'.format(img_name, img.shape))
ax = show_image(img, ax=ax)
for ann in anns:
    bb = ann['bbox']
    bb = ltlg2xywh(bb, mBB, img.shape[:2])
    draw_rect(ax, bb, 'green')

## Patch generation

In order to train our detector, we first generate patches from the original images. To that end we build a dataframe with the images and annotations. We save annotations in the COCO format (x, y, w, h).

In [ ]:
import pandas as pd

img_names, annotations = [], []
for ann in annotations_file:
    img_name = ann["name"]
    img_names.append(img_name)
    img = open_image('{}/{}'.format(PATH,img_name))
    anns = ann['annotations']
    mBB = ann['mapBB']
    _annotations = []
    for _ann in anns:
        bb = _ann['bbox']
        # convert bb from (lat, lng) to (x, y, w, h)
        bb = ltlg2xywh(bb, mBB, img.shape[:2])
        bb = [int(b) for b in bb]
        # save tuple (bb, label)
        _annotations.append((bb, 0))
    annotations.append(_annotations)
        
df = pd.DataFrame({'img_name': img_names, 'annotations': annotations})
df.head()

Now we split our dataset in training and validation images. During the training we will feed the network with batches of randomized patches from the trainig set. Then, we will use the patches in the validation set to evaluate the performance of the network.

In [ ]:
from sklearn.model_selection import train_test_split

df_t, df_v = train_test_split(df, test_size=0.2, shuffle=False, random_state=42)

print("Training images: ", len(df_t))
print("Validation images: ", len(df_v))

Choose a window and a stride and generate patches from images.

In [ ]:
# define a window and a stride and create folder to save patches

window, stride = 256, 100

patches_folder = '{}/patches_{}_{}'.format(PATH, window, stride)
if os.path.isdir(patches_folder):
    shutil.rmtree(patches_folder)
os.mkdir(patches_folder)

!ls {PATH} 

In [ ]:
# generate training patches

dfa, dfm = build_patches(df_t, PATH, patches_folder, window, stride, perc=0.5)
dfa.to_csv('{}/annotations_train.csv'.format(patches_folder), index=None)
dfm.to_csv('{}/mosaics_train.csv'.format(patches_folder), index=None)

In [ ]:
# generate validations patches

dfa, dfm = build_patches(df_v, PATH, patches_folder, window, stride, perc=0.5)
dfa.to_csv('{}/annotations_eval.csv'.format(patches_folder), index=None)
dfm.to_csv('{}/mosaics_eval.csv'.format(patches_folder), index=None)

Now we load our patch-level annotations to check that everything is ok. 

In [ ]:
# load annotations and visualize patches

df_t = pd.read_csv('{}/annotations_train.csv'.format(patches_folder))
mosaics_t = pd.read_csv("{}/mosaics_train.csv".format(patches_folder))

df_v = pd.read_csv('{}/annotations_eval.csv'.format(patches_folder))
mosaics_v = pd.read_csv("{}/mosaics_train.csv".format(patches_folder))

# convert string of bbs into list of bbs

df_t.annotations = anns_str2int(df_t.annotations.values)
mosaics_t.annotations = anns_str2int(mosaics_t.annotations.values)

df_v.annotations = anns_str2int(df_v.annotations.values)
mosaics_v.annotations = anns_str2int(mosaics_v.annotations.values)

print("Training patches: ",len(df_t))
print("Validation patches: ",len(df_v))

In [ ]:
# choose one dataset for visualization

df = df_t
mosaics = mosaics_t

In [ ]:
# show random patches

fig, axs = plt.subplots(3, 3, figsize=(10,10))
for i, _ax in enumerate(axs):
    for ix, ax in enumerate(_ax):
        
        ix = random.randint(0, len(df)-1)
        img_name, anns = df.loc[ix].img_name, df.loc[ix].annotations
        img = open_image('{}/{}'.format(patches_folder,img_name))
        ax = show_image(img, ax=ax)
        for bb, label in anns:
            draw_rect(ax, bb, 'green')

In [ ]:
# show patches in random mosaic

ix = random.randint(0, len(mosaics)-1)
mosaic = mosaics.loc[ix]

img_ori = open_image("{}/{}".format(PATH, mosaic.img_name))
anns_ori = mosaic.annotations
mosaic = mosaic.mosaic
mosaic = mosaic.split(',')[:-1]
mosaic = [m.split(' ')[:-1] for m in mosaic]
shape = (len(mosaic), len(mosaic[0]))

fig, axs = plt.subplots(shape[0], shape[1], figsize=(20,20))
for i, _ax in enumerate(axs):
    for j, ax in enumerate(_ax):

        ix = int(mosaic[i][j])
        img_name, anns = df.loc[ix].img_name, df.loc[ix].annotations
        img = open_image('{}/{}'.format(patches_folder,img_name))
        ax = show_image(img, ax=ax)
        for bb, label in anns:
            draw_rect(ax, bb, 'green')

In [ ]:
# show original image

fig, ax = plt.subplots(figsize=(10,10))
ax = show_image(img_ori, ax=ax)
for bb, label in anns_ori:
    draw_rect(ax, bb, 'green')

With the data in place, we can proceed to the training.